In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt

##### Import du json

In [ ]:
# Charger les résultats existants
with open("siren_attachments.json", "r") as f:
    results = json.load(f)

In [ ]:
# Vérification des keys et des données pour un SIREN
siren, data = next(iter(results.items()))
print(siren)
print(data.keys())


---

##### Visu rapide du json en dataframe façon bilan comptable

In [ ]:
rows = []

for siren, data in results.items():
    for bilan in data.get("bilans", []):
        dateDepot = bilan.get("dateDepot")
        typeBilan = bilan.get("typeBilan")
        numChrono = bilan.get("numChrono")
        
        # Vérifier si le bilan contient un bilan saisi
        bilan_saisi = bilan.get("bilanSaisi")
        if bilan_saisi:
            pages = bilan_saisi.get("bilan", {}).get("detail", {}).get("pages", [])
            for page in pages:
                liasses = page.get("liasses", [])
                for ligne in liasses:
                    rows.append({
                        "siren": siren,
                        "dateDepot": dateDepot,
                        "typeBilan": typeBilan,
                        "numChrono": numChrono,
                        "code": ligne.get("code"),
                        "m1": ligne.get("m1"),
                        "m2": ligne.get("m2"),
                        "m3": ligne.get("m3"),
                        "m4": ligne.get("m4"),
                    })

df = pd.DataFrame(rows)
print(df.head())
print("\nNombre total de lignes:", len(df))

In [ ]:
df.shape

In [ ]:
df.dtypes

##### Vérification du nombre de bilans par structure

In [ ]:
pd.set_option("display.max_rows", None)
# Identifier les bilans uniques par SIREN
bilan_unique = df.drop_duplicates(subset=["siren", "numChrono"])

# Compter combien de bilans par SIREN
bilan_counts = bilan_unique["siren"].value_counts()

# Afficher un aperçu
print(bilan_counts.head(50))


In [ ]:
len(bilan_counts)

In [ ]:
# Choisir un SIREN et un bilan précis
siren_cible = "831071121"
num_chrono_cible = "B2019/025624"

# Filtrer toutes les liasses de ce bilan
bilan_complet = df[(df["siren"] == siren_cible) & (df["numChrono"] == num_chrono_cible)]

# Afficher
print(bilan_complet)


In [ ]:
# Série avec le nombre de bilans par SIREN
nb_bilans_par_siren = df.groupby("siren")["numChrono"].nunique()

# Histogramme
plt.figure(figsize=(10,6))
nb_bilans_par_siren.value_counts().sort_index().plot(kind="bar")
plt.xlabel("Nombre de bilans récupérés par SIREN")
plt.ylabel("Nombre de SIREN")
plt.title("Répartition du nombre de bilans par SIREN")
plt.xticks(rotation=0)
plt.show()


In [ ]:
# nb_bilans_par_siren : nombre de bilans uniques par SIREN
nb_bilans_par_siren = df.groupby("siren")["numChrono"].nunique()

# Créer un DataFrame résumé
resume_bilans = nb_bilans_par_siren.reset_index().rename(columns={"numChrono": "nb_bilans"})
resume_bilans["etat"] = resume_bilans["nb_bilans"].apply(lambda x: "aucun bilan" if x == 0 else "bilans récupérés")

# Aperçu
print(resume_bilans.head(20))
print("\nDistribution des bilans :")
print(resume_bilans["etat"].value_counts())

In [ ]:
# SIREN qui ont un résultat vide → probablement touchés par 429 ou autres erreurs
siren_vides = [s for s, v in results.items() if v == {}]

print(f"Nombre de SIREN vides : {len(siren_vides)}")
print("Exemples :", siren_vides[:20])

In [ ]:
# Créer une liste de tous les SIREN
all_siren = list(results.keys())

# Construire un DataFrame
audit = pd.DataFrame({
    "siren": all_siren,
})

# Nombre de bilans récupérés
audit["nb_bilans"] = audit["siren"].apply(lambda x: len(results[x].get("bilans", [])))

# État
audit["etat"] = audit["nb_bilans"].apply(lambda x: "bilans récupérés" if x > 0 else "aucun bilan")

# Probable 429 si aucun bilan (ou comptesResultats) récupéré
audit["probable_429"] = audit["siren"].apply(
    lambda x: True if not results[x].get("bilans") and not results[x].get("comptesResultats") else False
)

# Tri pour voir d’abord ceux probablement touchés par 429
audit_sorted = audit.sort_values(by="probable_429", ascending=False)

# Aperçu
print(audit_sorted.head(50))

---

#### Vérification des SIREN touchés potentiellement par le 429

In [ ]:
nb_probable_429 = audit["probable_429"].sum()
print(f"Nombre de SIREN probablement touchés par un 429 : {nb_probable_429}")


In [ ]:
siren_avec_bilan = (audit["nb_bilans"] > 0).sum()
print(f"Nombre de SIREN avec au moins un bilan : {siren_avec_bilan}")


---

In [ ]:
# Filtrer uniquement les SIREN avec au moins un bilan
df_avec_bilan = df[df['siren'].isin(audit.loc[audit['nb_bilans'] > 0, 'siren'])]

# Prendre 10 bilans uniques (SIREN + numChrono)
bilans_uniques = df_avec_bilan.drop_duplicates(subset=['siren', 'numChrono'])
bilans_uniques.head(10)


In [ ]:
# Relance à prévoir des SIREN probablement touchés par un 429
siren_a_relancer = audit.loc[audit["probable_429"], "siren"].tolist()
print(f"Exemple de SIREN à relancer : {siren_a_relancer[:20]}")


---

#### Reconstitution des bilans

In [ ]:
bilans_uniques.shape